<div style="text-align: center;">
<a target="_blank" href="https://colab.research.google.com/github/bmalcover/aa_2425/blob/main/10_VGG_Custom_Dataset/VGG_Custom.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
</div>

In [6]:
import os

import torch
import torch.optim as optim
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

import glob as glob
import cv2

# Introducció

En aquesta pràctica, treballarem amb un conjunt de dades d'imatges de cans i moixos que ja heu utilitzat a la pràctica de la part anterior de l'assignatura. Tot i que coneixem aquest conjunt de dades, en aquesta ocasió ho em d'emprar amb ``Pytorch`` per tant haurem d'adaptar com ho llegim: fent un pre-processat de les dades per aplicar-los nous models d'aprenentatge profund.

L'objectiu de la sessió serà experimentar amb les diferents versions de la xarxa de convolució ``VGG``, com ara VGG16 i VGG19. Aquesta pràctica us permetrà veure com canvien els resultats d'entrenament i predicció en funció de l'arquitectura emprada, i reflexionar sobre els avantatges i inconvenients d'incrementar la profunditat d'una xarxa de convolució.

# Preparam les dades
Primerament descarregam les dades en el Google Colab. Per tal de fer-ho emprarem les eines ``wget`` i ``unzip``.


In [ ]:
!wget https://github.com/bmalcover/aa_2425/releases/download/v1/gatigos.zip
!unzip gatigos.zip

# *Custom dataset*

Un dataset personalitzat en PyTorch permet preparar i gestionar dades específiques per a un projecte d’aprenentatge automàtic. La classe ``Dataset`` de ``PyTorch`` és la base per crear aquest tipus de dataset i requereix la implementació de tres mètodes essencials: ``__init__``, ``__len__``, i ``__getitem__``.

* ``__init__``: Aquest mètode inicialitza el ``dataset`` i defineix els paràmetres que seran necessaris, com ara la ubicació de les dades o qualsevol transformació a aplicar. Aquí es poden carregar rutes d'imatges o etiquetes i definir les transformacions que es realitzaran.

* ``__len__``: Aquest mètode retorna el nombre total d'exemples en el ``dataset``. ``PyTorch`` l'utilitza per saber quantes mostres conté el conjunt de dades, cosa que és essencial per crear les batchs d’entrenament.
* ``__getitem__``: Aquest mètode accedeix a una mostra concreta del ``dataset``, identificada per un índex, i retorna les dades i la seva etiqueta (o ``target``). Normalment, s’apliquen les transformacions aquí abans de retornar la mostra, per assegurar que cada dada té el format adequat per al model.

Un cop creada, aquesta classe es pot emprar amb el ``DataLoader`` de ``PyTorch`` per gestionar l'entrenament en *batchs*, fent que el dataset personalitzat sigui eficient i fàcil de treballar dins del flux d’aprenentatge automàtic de ``PyTorch``.


In [43]:
class CatIGosDataset(Dataset):
    """Cat i Gos dataset."""

    def __init__(self, transform=None):
        """

        Args:
            transform:
        """
        self.image_paths = glob.glob("./**/*.png")
        self.xml_paths = glob.glob("./**/*.xml")


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        annotation = self.get_dict_xml(idx)
        return image, annotation

    def get_dict_xml(self, idx):
        # Parse XML and extract class name
        xml_path = self.xml_paths[idx]
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Extract the class name from <object> -> <name>
        class_name = None
        for obj in root.findall('object'):
            name = obj.find('name')
            if name is not None:
                class_name = name.text
                break  # Get the first class name found

        return {
            "path": xml_path,
            "output": class_name
        }


In [46]:
class CatIGosDataset(Dataset):
    """Cat i Gos dataset."""

    def extract_xml_annotation(self, imagenes, path):
      labels={}
      for path_img in imagenes:
        _,name = os.path.split(path_img)
        name = name.split(".")[0]

        name_xml = path+f"/annotations/{name}.xml"
        tree = ET.parse(name_xml)
        root = tree.getroot()

        annotation = root.find('object').find('name').text
        annotation = 0 if annotation == 'cat' else 1

        labels[path_img] = annotation
      return labels

    def __init__(self, path, transform=None):
      self.imagenes = glob.glob(path+"/images/*.png")
      self.labels = self.extract_xml_annotation(self.imagenes, path)


    def __len__(self):
        return len(self.imagenes)

    def __getitem__(self, idx):
        img = cv2.imread(self.imagenes[idx])
        label = self.labels[self.imagenes[idx]]
        return img, label

In [47]:
jorge = CatIGosDataset("./")
# jorge = CatIGosDataset()

a, b = jorge.__getitem__(7)

print(a)
print(b)

[[[105  90  94]
  [ 99  85  89]
  [ 96  82  86]
  ...
  [111 110 112]
  [115 108 111]
  [115 104 107]]

 [[108  93  97]
  [105  91  95]
  [103  89  93]
  ...
  [114 111 113]
  [123 114 117]
  [119 105 109]]

 [[103  88  92]
  [104  90  94]
  [104  90  94]
  ...
  [120 113 116]
  [128 117 120]
  [120 105 109]]

 ...

 [[173 174 172]
  [153 154 152]
  [162 163 161]
  ...
  [213 217 205]
  [210 214 202]
  [203 207 195]]

 [[172 173 171]
  [162 163 161]
  [172 173 171]
  ...
  [201 205 193]
  [197 201 189]
  [206 210 198]]

 [[173 174 172]
  [168 169 167]
  [177 178 176]
  ...
  [212 216 204]
  [207 211 199]
  [216 220 208]]]
1


In [50]:
BATCH_SIZE = 4
EPOCHS = 5

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#dataset = CatIGosDataset("./")

#train = dataset.ImageFolder('../data/tiny-imagenet/train', transform=transform)
#test = dataset.ImageFolder('../data/tiny-imagenet/test', transform=transform)

imagenes = glob.glob("CatIGos/images/*.png")

train, test = train_test_split(imagenes, test_size=0.34, random_state=42)

datasetTrain = CatIGosDataset('./CatIGos',images=train, transform=transform)
datasetTest = CatIGosDataset('./CatIGos',images=test, transform=transform)

train_loader = torch.utils.data.DataLoader(datasetTrain,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(datasetTest,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

ValueError: With n_samples=0, test_size=0.34 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

# Definicio de la xarxa: VGG i *Transfer learning*

En aquesta pràctica aplicarem la tècnica de transfer learning amb una de les xarxes CNN més conegudes i profundes:

 - VGG. [Very Deep Convolutional Networks for Large-Scale Image Recognition, 2014](https://arxiv.org/abs/1409.1556). La mida d'entrada de les imatges és de (224x224x3). VGG es presenta en diferents variants, com ara VGG16 i VGG19, que contenen respectivament 16 i 19 capes amb aproximadament 138 milions de paràmetres entrenables en el cas de VGG16.

Descarregarem VGG i l'analitzarem. En aquest cas, no només obtenim la seva arquitectura, sinó també els pesos resultants del seu entrenament en grans conjunts de dades.





In [ ]:
vgg11 = models.vgg11(weights=True)

print("-" * 50)
print("Arquitectura VGG11")
print("-" * 50)
print(vgg11)

## Com emprar la GPU per entrenar un model

Un dels elements diferencials d'aquest model, respecte als que havíem vist fins ara, és la seva mida i, per tant, l'entrenament es torna impossible emprant __CPU__ directament. Per resoldre-ho hem d'emprar una **GPU**, a Google Colab disposam d'elles gratuïtament. Per fer-ho amb *Pytorch* hem de fer tres passes:

1. Comprovar que hi ha una GPU disponible.
2. Moure el model a GPU.
3. Moure les dades a GPU.

### Comprova si tenim una GPU disponible

Primer de tot, cal verificar si hi ha una GPU disponible a l’entorn. Això es pot fer amb el següent codi:

```python

import torch

is_cuda = torch.cuda.is_available()
```

Si la variable ``is_cuda`` és certa, llavors tens accés a una GPU.

### Mou el model a la GPU

En PyTorch, els models han d'estar explícitament en la GPU per poder fer servir la seva potència de càlcul. Si estàs carregant un model preentrenat (com AlexNet, ResNet, etc.), o si has definit el teu propi model, pots moure’l a la GPU amb ``.to(device)``, on device fa referència a la GPU.

```python

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
```

Això mou el model a la GPU (si està disponible). Si només tens una CPU, el model es mantindrà a la CPU.

### Mou les dades a la GPU

No només el model, sinó que també les dades (inputs) han d'estar a la GPU per fer les operacions més ràpides. Així, abans de fer servir les dades com a inputs del model, assegura't de moure-les al mateix device:

```python

# Exemple d'un batch de dades
inputs, labels = inputs.to(device), labels.to(device)
```


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vgg11.to(device)

# Entrenament

In [ ]:
# TODO


## Feina a fer:

1. Preparar el *dataset* personalitzat.
2. Carregar la xarxa VGG11, VGG16 i VGG19, amb i sense batch normalization.
3. Entrenar-ho fent *transfer learning*.
4. Comparar els resultats.
